In [ ]:
'''Ant colony optimisation. The choice of the cell to move to will depend on the distance between cells (attractivity) and pheromone level. Ants only decide on cells that are reachable. This gives a probability of moving from one cell to each neighbouring cell. And based on this the agent can sample and select one.
Each cell has a probability of the ant transitioning to it and it needs to randomly select which one it goes to using a  weighted random.choice = doesn't have to be the optimal path, just a path using ant colony'''

In [ ]:
'''It should be stressed that the Ant Colony Optimization has been constructed to seek solutions of NP-hardproblems. As such, there is thus no guarantee that the most optimumsolution will be always found. Therefore, the obtained results may be bothoptimal (accurate) and approximations that depend on the degree of fitnessof the algorithm itself for each individual problem to be solved.
Thanks to a construction ofthe shortest path tree it is not necessary to find the shortest paths from thesource to all of the nodes one by one, but only to such nodes that have notyet been included in the tree.

The number of ants influences considerably the accuracy of a solution obtained as a result of the operation of the algorithm. At the same time, it increases time of its operation.

Ant  colony  optimization  is  an  iterative  algorithm.  At  eachiteration,  a  number  of  artificial  ants  are  considered.  Each  ofthem builds a solution by walking from vertex to vertex on thegraph with the constraint of not visiting any vertex that she hasalready  visited  in  her  walk.  At  each  step  of  the  solution  con-struction,  an  ant  selects  the  following  vertex  to  be  visitedaccording  to  a  stochastic  mechanism  that  is  biased  by  thepheromone: when in vertex i, the following vertex is selectedstochastically  among  the  previously  unvisited  ones  (see  Figure2). In particular, if jhas not been previously visited, it can beselected with a probability that is proportional to thepheromone associated with edge (i,j).

VERTICES ARE NODES, EDGES ARE TIME VALUES/DISTANCE BETWEEN NODES, COORDINATES OF CELLS ARE NODES
MEW REPRESENTS ATTRACTIVENESS OF A CELL WHICH IS 1/TIMEVALUES SO 1/MAZE
'''

In [1]:
import math
import matplotlib.pyplot as plt
import random
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

In [2]:
height, width = 3, 3
maze = np.array(np.random.randint(0, 10, size = (height, width)))

In [3]:
maze

array([[4, 0, 9],
       [5, 9, 6],
       [2, 8, 8]])

In [4]:
maze[0][0] 

4

In [24]:
def fill_initial_pheromones(Lij): # creates a 2D pheromone array where Lij is the original maze with time values. This sets the pheromones to be equal to the attractiveness/desirability of the vertex
    pheromones = 1/Lij
    pheromones[pheromones==1] = 0.65 
    pheromones[pheromones>1] = 0.70 # 1/0 becomes infinity so we replace these with 1s and to scale it, if a cell has distance 1 then it has 0.75
    return pheromones

In [25]:
initial_pheromones = fill_initial_pheromones(maze)
pheromones = fill_initial_pheromones(maze) # initial pheromone concentration resembles the attractivity of each cell, use this for attractiveness and use nodes for distances
move_attractiveness = fill_initial_pheromones(maze)

In [26]:
initial_pheromones

array([[0.25      , 0.7       , 0.11111111],
       [0.2       , 0.11111111, 0.16666667],
       [0.5       , 0.125     , 0.125     ]])

In [27]:
pheromones

array([[0.25      , 0.7       , 0.11111111],
       [0.2       , 0.11111111, 0.16666667],
       [0.5       , 0.125     , 0.125     ]])

In [28]:
move_attractiveness

array([[0.25      , 0.7       , 0.11111111],
       [0.2       , 0.11111111, 0.16666667],
       [0.5       , 0.125     , 0.125     ]])

In [29]:
current_ant_loc = [0,0]

In [30]:
path_of_selected_ant = []

In [31]:
path_of_selected_ant.append(maze[current_ant_loc[0]][current_ant_loc[1]])

In [32]:
path_of_selected_ant

[4]

In [33]:
path_of_selected_ant_coords = []

In [34]:
path_of_selected_ant_coords.append(current_ant_loc) 

In [35]:
path_of_selected_ant_coords

[[0, 0]]

In [36]:
def get_neighbours(maze, current_ant_loc): # returns a list of accessible nodes to the ant, THIS VERSION USES maze with nodes as vertices
    neighbours = [] # contains the up, down, left, right accessible nodes for the ants
   #if current_node_location contains 0 or positive numbers then append to neighbours else append None?
    if current_ant_loc[0]-1 >= 0:
        neighbours.append(maze[current_ant_loc[0]-1][current_ant_loc[1]])
    else: neighbours.append(np.NaN)
    if current_ant_loc[0]+1 <= height-1:
        neighbours.append(maze[current_ant_loc[0]+1][current_ant_loc[1]])
    else: neighbours.append(np.NaN)
    if current_ant_loc[1]-1 >= 0:
        neighbours.append(maze[current_ant_loc[0]][current_ant_loc[1]-1])
    else: neighbours.append(np.NaN)
    if current_ant_loc[1]+1 <= width-1:
        neighbours.append(maze[current_ant_loc[0]][current_ant_loc[1]+1])
    else: neighbours.append(np.NaN)
    return neighbours

In [37]:
neighbours = np.array(get_neighbours(maze, current_ant_loc))
neighbours

array([nan,  5., nan,  0.])

In [38]:
def get_neighbours_coords(maze, current_ant_loc): # returns a list of accessible cells to the ant, THIS VERSION USES maze with time values and their coordinates in the 2D array as vertices - NEEDED FOR GRAPH
    neighbours_coords = [] # contains the up, down, left, right accessible nodes for the ants
   #if current_node_location contains 0 or positive numbers then append to neighbours else append None?
    if current_ant_loc[0]-1 >= 0:
        neighbours_coords.append([current_ant_loc[0]-1, current_ant_loc[1]])
    else: neighbours_coords.append('Not Possible')
    if current_ant_loc[0]+1 <= height-1:
        neighbours_coords.append([current_ant_loc[0]+1, current_ant_loc[1]])
    else: neighbours_coords.append('Not Possible')
    if current_ant_loc[1]-1 >= 0:
        neighbours_coords.append([current_ant_loc[0], current_ant_loc[1]-1])
    else: neighbours_coords.append('Not Possible')
    if current_ant_loc[1]+1 <= width-1:
        neighbours_coords.append([current_ant_loc[0], current_ant_loc[1]+1])
    else: neighbours_coords.append('Not Possible')
    return neighbours_coords

In [39]:
neighbours_coords = np.array(get_neighbours_coords(maze, current_ant_loc))
neighbours_coords

array(['Not Possible', list([1, 0]), 'Not Possible', list([0, 1])],
      dtype=object)

In [40]:
pheromone_neighbours = np.array(get_neighbours(pheromones, current_ant_loc))
pheromone_neighbours

array([nan, 0.2, nan, 0.7])

In [ ]:
        '''
        for i in path_of_selected_ant_coords:
            if i == np.any(pheromone_neighbours_coords):
                index_of_previous_ant_location = 
                pheromone_neighbours[index_of_previous_ant_location] = 0.0
        for i in range(len(pheromone_neighbours_coords)):
            if pheromone_neighbours_coords[i][0] == path_of_selected_ant_coords[-1][0] and pheromone_neighbours_coords[i][1] == path_of_selected_ant_coords[-1][1]:
                index_of_previous_ant_location = i
                pheromone_neighbours[index_of_previous_ant_location] = 0.0                                 # set the probability of moving to the previous ant location = 0
        '''

In [42]:
maze

array([[4, 0, 9],
       [5, 9, 6],
       [2, 8, 8]])

In [43]:
def determine_Pij_neighbours(pheromones, move_attractiveness, current_ant_loc, path_of_selected_ant_coords, alpha, beta):
    pheromone_neighbours = np.array(get_neighbours(pheromones, current_ant_loc))
    attractiveness_neighbours = np.array(get_neighbours(move_attractiveness, current_ant_loc))
    
    pheromone_neighbours_coords = np.array(get_neighbours_coords(pheromones, current_ant_loc))
    attractiveness_neighbours_coords = np.array(get_neighbours_coords(move_attractiveness, current_ant_loc))
    '''
    if len(path_of_selected_ant_coords) > 1:
        for i in range(len(pheromone_neighbours_coords)):
            if pheromone_neighbours_coords[i][0] == path_of_selected_ant_coords[-1][0] and pheromone_neighbours_coords[i][1] == path_of_selected_ant_coords[-1][1]:
                index_of_previous_ant_location = i
                pheromone_neighbours[index_of_previous_ant_location] = 0.0                                 # set the probability of moving to the previous ant location = 0
        #for i in range(len(pheromone_neighbours_coords)):
        #    for j in range(len(path_of_selected_ant_coords)):
        #        if pheromone_neighbours_coords[i][0] == path_of_selected_ant_coords[j][0] and pheromone_neighbours_coords[i][1] == path_of_selected_ant_coords[j][1]:
        #            pheromone_neighbours[i] = 0.0        
    '''
    pheromone_times_attractiveness = np.multiply(np.power(pheromone_neighbours, alpha), np.power(attractiveness_neighbours, beta))

    np.nan_to_num(pheromone_times_attractiveness, copy=False, nan=0)                # makes the probability of inaccessible nodes 0
    for i in range(len(pheromone_neighbours_coords)):
        for j in range(len(path_of_selected_ant_coords)):
            if pheromone_neighbours_coords[i][0] == path_of_selected_ant_coords[j][0] and pheromone_neighbours_coords[i][1] == path_of_selected_ant_coords[j][1]:
                pheromone_times_attractiveness[i] = 0                                                                           # if the chosen cell is in the path already traversed, probability = 0
    Pij = pheromone_times_attractiveness/np.sum(pheromone_times_attractiveness)                                                 # gives probabilities of choosing a cell to move to
    return Pij

In [44]:
neighbour_Pij = determine_Pij_neighbours(pheromones, move_attractiveness, current_ant_loc, path_of_selected_ant_coords, alpha=1, beta=2)
neighbour_Pij

array([0.        , 0.02279202, 0.        , 0.97720798])

In [545]:
p = [[0, 0], [1, 0], [1, 1], [2, 1], [2, 0]]
n = [[1, 0], 'Not Possible', 'Not Possible', [2,1]]
for i in range(len(n)):
    for j in range(len(p)):
        if n[i][0] == p[j][0] and n[i][1] == p[j][1]:
            print(i)

0
3


In [46]:
current_ant_loc

[0, 0]

In [47]:
path_of_selected_ant_coords

[[0, 0]]

In [48]:
np.array(get_neighbours_coords(pheromones, current_ant_loc))

array(['Not Possible', list([1, 0]), 'Not Possible', list([0, 1])],
      dtype=object)

In [49]:
maze

array([[4, 0, 9],
       [5, 9, 6],
       [2, 8, 8]])

In [53]:
np.count_nonzero(neighbour_Pij)


2

In [ ]:
p = [0, 1, 2 ,3
a = 

In [ ]:
neighbours_coords[np.random.choice(neighbours_coords.shape[0], np.count_nonzero(neighbour_Pij), False, neighbour_Pij)]

In [ ]:
neighbours_coords[np.random.choice(neighbours_coords.shape[0], np.count_nonzero(neighbour_Pij), False, neighbour_Pij)]

In [68]:
neighbours_coords

array([[0, 1],
       [2, 1],
       [1, 0],
       [1, 2]])

In [77]:
rnd_indices = np.random.choice(neighbours_coords.shape[0], np.count_nonzero(neighbour_Pij), False, neighbour_Pij)
possible_choice_cell_coords = neighbours_coords[rnd_indices][0]
print(possible_choice_cell_coords)
rnd_indices2 = np.random.choice(possible_choice_cell_coords.shape[0], 1, False)
choice_cell_coords = possible_choice_cell_coords[rnd_indices2][0]
print(choice_cell_coords[0])

[0 1]


IndexError: invalid index to scalar variable.

In [64]:
''' In ACS, the choice of the next node to visit is based on random choice but with the priority of better Pij values for this reason transition probability calculation is changed a bit to get value in the range 0-100%.'''
def ant_move(neighbour_Pij, neighbours_coords):
  possible_choice_cell_coords = neighbours_coords[np.random.choice(neighbours_coords.shape[0], np.count_nonzero(neighbour_Pij), False, neighbour_Pij)]
  choice_cell_coords = np.random.choice(possible_choice_cell_coords, 1, False)[0]
  for i in range(len(path_of_selected_ant_coords)):
    if choice_cell_coords[0] == path_of_selected_ant_coords[i][0] and choice_cell_coords[1] == path_of_selected_ant_coords[i][1]:
      while choice_cell_coords[0] == path_of_selected_ant_coords[i][0] and choice_cell_coords[1] == path_of_selected_ant_coords[i][1]:
          choice_cell_coords = np.random.choice(possible_choice_cell_coords, 1, False)[0]  
  new_ant_loc = choice_cell_coords
  return choice_cell_coords, new_ant_loc

  #if choice_cell_coords in path_of_selected_ant_coords:
  #  while choice_cell_coords in path_of_selected_ant_coords:
  #    choice_cell_coords = neighbours_coords[np.random.choice(neighbours_coords.shape[0], 1, False, neighbour_Pij)][0]
  #new_ant_loc = choice_cell_coords
  #choice_cell_coords = neighbours_coords[np.random.choice(neighbours_coords.shape[0], 1, False, neighbour_Pij)][0] # returns the coords of the chosen cell - CAN BE USED WITH NODE_MAZE AND MAZE TO IDENTIFY LOCATION
  #new_ant_loc = choice_cell_coords # update current ant location
  
#choice_cell = np.random.choice(neighbours, 1, True, neighbour_Pij)
#choice_node = list(np.random.choice(neighbours_nodes, 1, True, neighbour_Pij))

In [ ]:
p = [[0, 0], [0, 1], [0, 2], [1, 2], [1, 1]]
n = [1,1]
pij = [0.1, 0.2, 0.6, 0.1]
n_coords = np.array([1,2,3,4])
choice_coord_ind = np.random.choice(n_coords.shape[0], 1, False, pij)
print(choice_coord_ind)
choice = n_coords[choice_coord_ind][0]
print(choice) # we don't like this choice for some reason
pij[choice_coord_ind[0]] = 0
choice_coord_ind2 = np.random.choice(n_coords.shape[0], 1, False, pij) #sorted(pij, reverse=True)[1]# choose second highest instead?
print(choice_coord_ind2)
choice2 = n_coords[choice_coord_ind2][0]
print(choice2)
#for i in range(len(p)):
 #   if n[0] == p[i][0] and n[1] == p[i][1]:
 #       while n[0] == p[i][0] and n[1] == p[i][1]:
 #           n = [1,3]
 #           print('no')
            

In [554]:
path_of_selected_ant_coords

[[0, 0]]

In [556]:
choice_cell_coords, current_ant_loc = ant_move(neighbour_Pij, neighbours_coords)
choice_cell_coords

[0, 1]

In [557]:
current_ant_loc

[0, 1]

In [558]:
path_of_selected_ant_coords

[[0, 0]]

In [58]:
rho = 0.2 # evaporation factor
def local_pheromone_update(pheromones, initial_pheromones, choice_cell_coords, rho): #new pheromone value at chosen cell/node
    pheromones[choice_cell_coords[0]][choice_cell_coords[1]] = (1.0-rho) * (pheromones[choice_cell_coords[0]][choice_cell_coords[1]]) + (rho * initial_pheromones[choice_cell_coords[0]][choice_cell_coords[1]])
    return pheromones

In [560]:
initial_pheromones

array([[0.16666667, 0.8       , 0.125     ],
       [0.16666667, 0.65      , 0.16666667],
       [0.16666667, 0.14285714, 0.2       ]])

In [561]:
pheromones = local_pheromone_update(pheromones, initial_pheromones, choice_cell_coords, rho)
pheromones

array([[0.16666667, 0.8       , 0.125     ],
       [0.16666667, 0.65      , 0.16666667],
       [0.16666667, 0.14285714, 0.2       ]])

In [562]:
path_of_selected_ant.append(maze[choice_cell_coords[0]][choice_cell_coords[1]])                         # contains the time values of the path the ant takes in this loop

In [563]:
path_of_selected_ant

[6, 0]

In [564]:
path_of_selected_ant_coords.append(current_ant_loc)                                                  # contains the indices of the path the ant takes in this loop

In [565]:
path_of_selected_ant_coords

[[0, 0], [0, 1]]

In [59]:
def global_pheromone_update(pheromones, shortest_path_coords_iter, shortest_path_iter, rho): # (1-rho)*tij + rho*delta(tij) where delta tij is the 1/(total distance travelled of best path)
    for i, j in shortest_path_coords_iter:
        pheromones[i][j] = (1.0-rho) * (pheromones[i][j]) + (rho * (1/(shortest_path_iter)))
    return pheromones

## Main Loops

In [ ]:
'''First ant starts at [0,0] and then randomly walks, locally changing the pheromone content of the board. Once it has found a path to [height-1, width-1] i.e. the bottom right of the board, the pheromone content of cells on this path are updated globally by a small amount
Second ant then finds a path by picking the next cell to travel to in such a way that the cell with high pheromone value are more likely to be chosen than the cell with lower pheromone. Due to the random factor, the ant may sometimes pick the path with lower pheromone which can occasionally produce better/shorter path. If the ant succeeds, then increase the pheromone value on that path so the cells on the path are more likely to be chosen. If the ant doesn't succeed, then abort the ant and start again.
Occasionally, reduce the pheromone on the whole board so less successful cells would get picked less and less.
repeat sending ants
constraints -> ends when ant finds path to bottom right cell, ants cannot go to previously visited cells'''

In [ ]:
'''Need to change Pij_neighbour to change eliminate all cells travelled, not just the last visited one'''

In [65]:
initial_pheromones = fill_initial_pheromones(maze)                      # initial pheromone concentration resembles the attractivity of each cell - this is a parameter used in the calculation of local pheromone update
pheromones = fill_initial_pheromones(maze)                              # the 2D array of pheromones at each edge that is changed by ants
move_attractiveness = fill_initial_pheromones(maze)                     # the attractivity of each node - stays the same
num_iterations = 2                                                      # number of iterations                    
num_ants = 5                                                            # number of ants running per iteration
alpha, beta = 1, 2                                                                          # alpha and beta control the relative importance of the pheromone vs move attractiveness
rho = 0.3                                                                                   # evaporation rate
end_condition = [height-1, width-1]                                                         # ant stops at the bottom-right cell
paths_taken_by_all_ants = {}                                                                # dictionary to hold all the paths each ant takes in the loop
paths_coords_taken_by_all_ants = {}                                                         # dictionary to hold all the indices of the path each ant takes in the loop
total_distance_by_all_ants = {}                                                             # dictionary to hold all the total times each ant takes    
for i in range(num_iterations):
    for i in range(num_ants):
        current_ant_loc = [0,0]
        path_of_selected_ant_coords = []                                                     
        path_of_selected_ant_coords.append(current_ant_loc)                                 # adds the starting location to the path
        path_of_selected_ant = []
        path_of_selected_ant.append(maze[current_ant_loc[0]][current_ant_loc[1]])           # adds the starting time value to the path
        
        while np.any(current_ant_loc != end_condition):                                                             # runs loop until ant reaches bottom right cell
            neighbours_coords = np.array(get_neighbours_coords(maze, current_ant_loc))                              # returns array of the indices of the neighbouring cells (up, down, left, right)
            pheromone_neighbours = np.array(get_neighbours(pheromones, current_ant_loc))
            attractiveness_neighbours = np.array(get_neighbours(move_attractiveness, current_ant_loc))
            neighbour_Pij = determine_Pij_neighbours(pheromones, move_attractiveness, current_ant_loc, 
                                path_of_selected_ant_coords, alpha, beta)                                           # returns array of all the probabilities of moving to a neighbour
            
            choice_cell_coords, current_ant_loc = ant_move(neighbour_Pij, neighbours_coords)                        # chooses a neighbour to move to using a weighted random choice and updates the current ant location
            pheromones = local_pheromone_update(pheromones, initial_pheromones, choice_cell_coords, rho)            # updates the pheromone level of the cell the ant has moved to.
            path_of_selected_ant.append(maze[choice_cell_coords[0]][choice_cell_coords[1]])                         # contains the time values of the path the ant takes in this loop
            path_of_selected_ant_coords.append(current_ant_loc)                                                     # contains the indices of the path the ant takes in this loop
        
        #current_ant_loc = [height-1, width-1]                                                                       # if loop exits then end condition is satisfied and current ant location is at end
        #path_of_selected_ant.append(maze[current_ant_loc[0]][current_ant_loc[1]])                                   # adding the time value of last cell
        #path_of_selected_ant_coords.append(current_ant_loc)
        distance_of_path_of_selected_ant = np.sum(path_of_selected_ant)
        total_distance_by_all_ants[i] = distance_of_path_of_selected_ant                                            # iterates over a dictionary to add the time each ant takes to get to bottom right cell
        paths_taken_by_all_ants[i] = path_of_selected_ant                                                           # iterates over a dictionary to add the path of each ant
        paths_coords_taken_by_all_ants[i] = path_of_selected_ant_coords                                             # iterates over a dictionary to add the path indices of each ant
    shortest_path_ind = min(total_distance_by_all_ants, key=total_distance_by_all_ants.get)                         # returns the index (key value) of the shortest path in the dictionary of all paths traversed
    shortest_path_iter = total_distance_by_all_ants[shortest_path_ind]                                              # returns the time of the shortest path for the iteration
    shortest_path_coords_iter = paths_coords_taken_by_all_ants[shortest_path_ind]                                   # returns the coords for the shortest path in the dictionary for the iteration                               
    pheromones = global_pheromone_update(pheromones, shortest_path_coords_iter, shortest_path_iter, rho)            # then do global pheromone update using iteration-best path 
#shortest_path_ind_overall = min(total_distance_by_all_ants, key=total_distance_by_all_ants.get)
##shortest_path = paths_coords_taken_by_all_ants[shortest_path_ind_overall]
#print(shortest_path)
    #print(pheromones)
    #print(shortest_path_coords_iter)
    #print(shortest_path_iter)
shortest_path = paths_coords_taken_by_all_ants[shortest_path_ind]
print(shortest_path)

ValueError: a must be 1-dimensional

In [86]:
path_coords_grid = []
for x, y in paths_coords_taken_by_all_ants[shortest_path_ind]: # this needs to convert a path to coordinates on the graph
    print(x, y)

0 0
0 1
1 1
0 1
0 0
0 1
0 0
1 0
2 0
2 1
2 2
2 2


In [85]:
maze

array([[2, 2, 7],
       [4, 3, 9],
       [2, 7, 2]])

In [ ]:
'''
If alpha=0, probability of moving to a cell is based purely on heuristics. Cells close to each other are chosen. Behaves similar to a greedy algorithm.
If beta=0, probability based on pheromone concentration. Sometimes can lead to localised search space.

## NODE PLOT 

In [4]:
def plot_maze_path(annotate=True, chosen_path=True): 
        c1 = np.array([0, width])
        c2 = np.array([height, 0]) 
        dim = len(c1) 
        x_pts = np.linspace(c1[0], c2[0], width)
        y_pts = np.linspace(c1[1], c2[1], height)
        Xv, Yv = np.meshgrid(x_pts, y_pts)
        numpts = width*height      
        node_array = np.zeros((numpts, 2), dtype=float)
        node_array[:,0] = np.reshape(Xv, numpts)
        node_array[:,1] = np.reshape(Yv, numpts)
        num_cells = int(width-1)*(height-1)
        connectivity = np.zeros((num_cells, int(2**dim)), dtype=int)
        rows, cols = height-1, width-1 
        for row in range(rows):
            for col in range(cols):
                num = width*row + col
                connectivity[cols*row + col] = [num+0, num+1, num+width, num+width+1]

        annotations = maze.flatten()
        X,Y = node_array.T
        fig = plt.figure(figsize=(width,height)) 
        ax = fig.add_subplot(111)
        ax.set_aspect('auto')
        plt.axis('off')
        plt.scatter(X,Y, marker='o', s=50, color='g', alpha=1.0)
        if height == width:
            plt.plot(Xv,Yv, linewidth=2, color='k', alpha=0.2)
            plt.plot(Yv,Xv, linewidth=2, color='k', alpha=0.2)
        if annotate:                    
            for i, pos in enumerate(node_array):
                plt.text(pos[0], pos[1],  str(annotations[i]), color='k', verticalalignment='bottom', horizontalalignment='right', fontsize='xx-large')
            
        nodes_coords = np.split(node_array, height)
        '''
        path_coords_grid = []
        for x, y in option_coords[chosen_path]: # this needs to convert a path to coordinates on the graph
            path_coords_grid.append(nodes_coords[x][y])
        head_length = 0.1
        if chosen_path:
            for i in range(len(path_coords_grid)-1): # plots vectors for the shortest path
                dx = path_coords_grid[i+1][0] - path_coords_grid[i][0]
                dy = path_coords_grid[i+1][1] - path_coords_grid[i][1]
                vec_ab = [dx,dy]
                vec_ab_magnitude = np.sqrt(dx**2+dy**2)
                dx = dx / vec_ab_magnitude
                dy = dy / vec_ab_magnitude
                vec_ab_magnitude = vec_ab_magnitude - head_length
                plt.arrow(path_coords_grid[i][0], path_coords_grid[i][1], vec_ab_magnitude*dx, vec_ab_magnitude*dy, head_width=0.1, head_length=0.2, color='red')'''
        plt.show(block=False)
        return node_array, nodes_coords, connectivity

In [ ]:
nodes_array, nodes_maze, connectivity  = plot_maze_path(annotate=True, chosen_path=True)

## NOT USED

In [ ]:
if len(path_of_selected_ant_coords) >= 1:
    checks = np.zeros(len(pheromone_neighbours_coords))
    for i in range(len(pheromone_neighbours_coords)):
        checks[i] = list(np.where(pheromone_neighbours_coords[i] == path_of_selected_ant_coords[-1]))[0]
        if checks[i] == path_of_selected_ant_coords[-1]:
            print(checks[i])
            index_of_previous_ant_location = i
        pheromone_neighbours[index_of_previous_ant_location] = 0.0           # set the probability of moving to the previous ant location = 0
        '''
        if pheromone_neighbours_coords[i] == path_of_selected_ant_coords[-1]: #and pheromone_neighbours_coords[i] == path_of_selected_ant_coords[-1]:
            #index_of_previous_ant_location = i 
    pheromone_neighbours[index_of_previous_ant_location] = 0.0           # set the probability of moving to the previous ant location = 0
    '''

In [68]:
def shortest_path_aco(graph, maze, start_node, end_node, n_ants, alpha, beta, rho, tau_min, tau_max, iteration_limit):
    #d = np.ones((height,width), dtype=float)*np.inf
    #pred = np.zeros((height,width), dtype=float)
    #return d, pred
#d, pred = shortest_path_aco(nodes_maze, maze, nodes_maze[0][0], nodes_maze[height-1][width-1], n_ants, alpha, beta, rho, tau_min, tau_max, iteration_limit)

def euclidean(nodes): # use node array/coords for distances
    distance = math.sqrt(pow(nodes[1] - nodes[1], 2) + pow(nodes[0] - nodes[0], 2))
    return distance

In [55]:
euclidean(node_coords[0][0], node_coords[2][2])

4.242640687119285

In [ ]:
fill_initial_pheromones(start_position=(0,0))

In [ ]:
def fill_initial_pheromones(start_position): # creates a 2D phreromone array
    pheromones = np.zeros((height,width), dtype=float)
    Lij = euclidean(node_coords[start_position[0],[0]]) #returns array with distances for each node 
    for i in pheromones:
        for j in i:
            pheromones[j] = 1/Lij
    return pheromones

In [ ]:
'''city_to_city_score = pheromone ** alpha * (1.0 / distance) ** beta
prob_of_going_to_city(i) = city_to_city_score(i) / sum_of_all_available_city_to_city_scores'''

In [ ]:
 def ant_move(self, neighbour_Pij, neighbours_coords, path_of_selected_ant_coords):
        #choice_cell_coords_ind = np.random.choice(neighbours_coords.shape[0], 1, False, neighbour_Pij)
        choice_cell_coords = neighbours_coords[np.random.choice(neighbours_coords.shape[0], 1, False, neighbour_Pij)][0]                      # returns the coords of the chosen cell
        for i in range(len(path_of_selected_ant_coords)):
            if choice_cell_coords[0] == path_of_selected_ant_coords[i][0] and choice_cell_coords[1] == path_of_selected_ant_coords[i][1]:     # checks to see if the chosen cell is in the path already traversed
        ##        #choice_cell_coords = neighbours_coords[choice_cell_coords_ind[1]]
                while choice_cell_coords[0] == path_of_selected_ant_coords[i][0] and choice_cell_coords[1] == path_of_selected_ant_coords[i][1]:
                   choice_cell_coords = neighbours_coords[np.random.choice(neighbours_coords.shape[0], 1, False, neighbour_Pij)][0]                    # if it is, then choose another move
        #        choice_cell_coords = neighbours_coords[np.random.choice(neighbours_coords.shape[0], 1, False, neighbour_Pij)][0]
        new_ant_loc = choice_cell_coords                                                                                                                # update ant location
        return choice_cell_coords, new_ant_loc
